In [24]:
import json
import csv
import os
import glob
import pandas as pd
import numpy as np

In [25]:
def read_csv_from_path(path):
    return pd.read_csv(path)

In [26]:
raw_data = read_csv_from_path('../../visualizations_src/Covid_Semantics1.0.adjudicated-2020-10-04T2314-Tags.csv')

In [27]:
raw_data.head()

,article_batch_name,article_number,article_filename,article_sha256,article_text_length,tua_group_uuid,tua_group_name,tua_batch_uuid,tua_batch_name,tua_batch_final,...,tua_uuid,namespace,topic_name,case_number,answer_uuid,extra,highlight_count,start_pos,end_pos,target_text
0,CovidArticles/2005CIAReportOnCoronavirus.txt,2005,2005CIAReportOnCoronavirus.txt,47990959103662e94e796d979018922afddc880fb4b867...,3885,3113cf9a-d876-49a7-845d-36472205d2a9,Covid_Semantics1.0.adjudicated,b471eb8f-686e-43dc-a620-506af7e3fb6f,Adjudicator Nick Adams HLTR source task 854 ar...,True,...,3d2ef8e4-6aa6-401d-988f-40612ce49f28,Semantics2020_03,Language,1,NaN,{},1,164,198,a retelling of the 2005 CIA report
1,CovidArticles/2005CIAReportOnCoronavirus.txt,2005,2005CIAReportOnCoronavirus.txt,47990959103662e94e796d979018922afddc880fb4b867...,3885,3113cf9a-d876-49a7-845d-36472205d2a9,Covid_Semantics1.0.adjudicated,b471eb8f-686e-43dc-a620-506af7e3fb6f,Adjudicator Nick Adams HLTR source task 854 ar...,True,...,090a112f-e19f-4f84-8058-28660c9f75ac,Semantics2020_03,Language,3,NaN,{},4,341,348,COVID19
2,CovidArticles/2005CIAReportOnCoronavirus.txt,2005,2005CIAReportOnCoronavirus.txt,47990959103662e94e796d979018922afddc880fb4b867...,3885,3113cf9a-d876-49a7-845d-36472205d2a9,Covid_Semantics1.0.adjudicated,b471eb8f-686e-43dc-a620-506af7e3fb6f,Adjudicator Nick Adams HLTR source task 854 ar...,True,...,090a112f-e19f-4f84-8058-28660c9f75ac,Semantics2020_03,Language,3,NaN,{},4,871,879,COVID-19
3,CovidArticles/2005CIAReportOnCoronavirus.txt,2005,2005CIAReportOnCoronavirus.txt,47990959103662e94e796d979018922afddc880fb4b867...,3885,3113cf9a-d876-49a7-845d-36472205d2a9,Covid_Semantics1.0.adjudicated,b471eb8f-686e-43dc-a620-506af7e3fb6f,Adjudicator Nick Adams HLTR source task 854 ar...,True,...,090a112f-e19f-4f84-8058-28660c9f75ac,Semantics2020_03,Language,3,NaN,{},4,259,261,\u201d.
4,CovidArticles/2005CIAReportOnCoronavirus.txt,2005,2005CIAReportOnCoronavirus.txt,47990959103662e94e796d979018922afddc880fb4b867...,3885,3113cf9a-d876-49a7-845d-36472205d2a9,Covid_Semantics1.0.adjudicated,b471eb8f-686e-43dc-a620-506af7e3fb6f,Adjudicator Nick Adams HLTR source task 854 ar...,True,...,090a112f-e19f-4f84-8058-28660c9f75ac,Semantics2020_03,Language,3,NaN,{},4,527,530,\u201d .


In [28]:
raw_data[['article_number', 'start_pos', 'end_pos']].head()

,article_number,start_pos,end_pos
0,2005,164,198
1,2005,341,348
2,2005,871,879
3,2005,259,261
4,2005,527,530


In [29]:
raw_data[['article_number', 'start_pos', 'end_pos']].apply(lambda x: list(range(x['start_pos'], x['end_pos'] + 1)), axis=1).head()

0    [164, 165, 166, 167, 168, 169, 170, 171, 172, ...
1             [341, 342, 343, 344, 345, 346, 347, 348]
2        [871, 872, 873, 874, 875, 876, 877, 878, 879]
3                                      [259, 260, 261]
4                                 [527, 528, 529, 530]
dtype: object

In [37]:
def simple_data_from_raw_data(raw_data):
    """
    Take article_number, start_pos, end_pos, and add Indices of Label in Article. Rename them if necesssary.
    Assumption: raw_data contains columns named article_number, start_pos, end_pos
    """
    simple_data = raw_data[['article_number', 'article_sha256', 'topic_name', 'start_pos', 'end_pos']]
    simple_data = simple_data.rename(columns = {'article_number': 'Article ID', 'article_sha256': 'Article sha256', 'topic_name': 'Credibility Indicator Category', 'start_pos': 'Start', 'end_pos': 'End'})
    simple_data['Indices of Label in Article'] = simple_data.apply(lambda x: list(range(x['Start'], x['End'] + 1)), axis=1)
    
    return simple_data

In [38]:
simple_data = simple_data_from_raw_data(raw_data)
simple_data.head()

,Article ID,Article sha256,Credibility Indicator Category,Start,End,Indices of Label in Article
0,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,164,198,"[164, 165, 166, 167, 168, 169, 170, 171, 172, ..."
1,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,341,348,"[341, 342, 343, 344, 345, 346, 347, 348]"
2,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,871,879,"[871, 872, 873, 874, 875, 876, 877, 878, 879]"
3,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,259,261,"[259, 260, 261]"
4,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,527,530,"[527, 528, 529, 530]"


In [39]:
simple_data['Article ID'].unique()

array([  2005, 100054, 100055, 100056, 100058, 100057, 100059, 100063,
       100066, 100062, 100067, 493801, 493800, 493804, 493799])

In [40]:
simple_data[simple_data['Article ID'] == 2005].head()

,Article ID,Article sha256,Credibility Indicator Category,Start,End,Indices of Label in Article
0,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,164,198,"[164, 165, 166, 167, 168, 169, 170, 171, 172, ..."
1,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,341,348,"[341, 342, 343, 344, 345, 346, 347, 348]"
2,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,871,879,"[871, 872, 873, 874, 875, 876, 877, 878, 879]"
3,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,259,261,"[259, 260, 261]"
4,2005,47990959103662e94e796d979018922afddc880fb4b867...,Language,527,530,"[527, 528, 529, 530]"


In [41]:
def convert_to_csv_per_article(simple_data):
    default_file_name = "../../visualizations_src/visualization_data/VisualizationData_"
    for article_id in simple_data['Article ID'].unique():
        file_name = default_file_name + str(article_id) + ".csv"
        sub_simple_data = simple_data[simple_data['Article ID'] == article_id]
        sub_simple_data.to_csv(file_name)

In [42]:
convert_to_csv_per_article(simple_data)